In [69]:
from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
import plotly.express as px
import os
import sys
import plotly.graph_objects as go 
dir_path = os.path.abspath('')
sys.path.append(dir_path + '/../')
from labbiofisica import Interpolazione, final_val

from scipy.optimize import curve_fit
from scipy.stats import norm
import scipy.stats as stats


In [70]:
def gaussian(x, amp, mu, sigma, fondo):   # funzione gaussiana
    return amp * norm.pdf(x,loc = mu,scale = sigma) + fondo

In [71]:
def read(file, s, m):    # funzione di lettura
    return np.loadtxt(file, unpack = True, skiprows = s, max_rows = m)

In [72]:
def fittiamo(x_data, y_data, p0 = [1, 1, 1, 1]):    # funzione fit e ottimizzazione
    popt, pcov = curve_fit(gaussian, x_data, y_data, p0)
    # amp_fit, mu_fit, sigma_fit, offset_fit = popt      giusto per capire cosa spara fuori popt
    return popt, pcov    

In [73]:
def plottiamo_plt(data):    # plot con pyplot
    x, y = data
    popt, pcov = fittiamo(*data)
    plt.plot(*data, label = "Dati con rumore", color = 'plum')
    plt.plot(x, gaussian(x, *popt), label = "Fit Gaussiano", linestyle = '--', color = 'purple')
    plt.legend()
    plt.xlabel("x")
    plt.ylabel("y")
    plt.title(f"amp = {popt[0]:.2f},  mu = {popt[1]:.2f},  sigma = {popt[2]:.2f},  fondo = {popt[3]:.2f}")
    plt.show()

In [74]:
def plottiamo_plotly(data):
    
    fig = go.Figure()
    X, Y = data
    popt, pcov = fittiamo(*data)
    
    fig.add_trace(go.Scatter(x = X, y = Y, mode = 'lines', line_color = 'plum', 
                             showlegend=False,hoverinfo='skip'))

    fig.add_trace(go.Scatter(x = X, y = gaussian(X, *popt), mode='lines',showlegend=False,
        marker = dict(color = 'purple')))

    fig.update_layout(
            yaxis_title = "Gray Value",
            xaxis_title = "Distance (μm)",
            # title='g<sup>2</sup>(τ) vs t',
            #xaxis = dict(range = [np.log10(0.0001), np.log10(0.010)]),
            #width = 800,
            #height = 600,
            plot_bgcolor = 'white',
            font = dict(
                #family="Courier New, monospace",
                size = 18,
                color = "Black"
            ),
                # yaxis=dict(range=[0, 0.06])
        )

    fig.update_xaxes(
        mirror = True,
        ticks = 'outside',
        showline = True,
        linecolor = 'black',
        gridcolor = 'lightgrey'
    )
    fig.update_yaxes(
        mirror = True,
        ticks = 'outside',
        showline = True,
        linecolor = 'black',
        gridcolor = 'lightgrey'
    )
    fig.show()
    return popt, pcov

In [75]:
def risoluz_olio():
    n_olio = 1.47
    NA = n_olio * np.sin(np.pi / 6)
    l_verde = 550
    risoluzione_attesa = ((1.22 * l_verde) / (2 * NA)) * 0.6
    print(risoluzione_attesa)

In [76]:
def FWHM(popt):
    print(f"FWHM = {2 * np.sqrt(2 * np.log(2)) * popt[2] * 1000:.2f}")

# caso 1

In [79]:
skiprows1 = 8
maxrows1 = 64

data1 = np.array(read(f'dati1_2.txt', skiprows1, maxrows1))
popt1, pcov1 = plottiamo_plotly(data1)  # scegliere plotly o pyplot
FWHM(popt1)

FWHM = 622.52


# caso 2

In [82]:
skiprows2 = 50
maxrows2 = 200

data2 = np.array(read(f'dati_verde_olio.txt', skiprows2, maxrows2))
popt2, pcov2 = plottiamo_plotly(data2)  # scegliere plotly o pyplot
FWHM(popt2)

FWHM = 229.13


# caso 3

In [88]:
skiprows3 = 120
maxrows3 = 300

data3 = np.array(read(f'dati_verde_olio2.txt', skiprows3, maxrows3))
popt3, pcov3 = plottiamo_plotly(data3)  # scegliere plotly o pyplot
FWHM(popt3)

FWHM = 225.29


# caso 4

In [100]:
skiprows4 = 150
maxrows4 = 250

data4 = np.array(read(f'dati_verde_olio2.txt', skiprows4, maxrows4))
popt4, pcov4 = plottiamo_plotly(data4)  # scegliere plotly o pyplot
FWHM(popt4)

FWHM = 221.51


# caso 5

In [106]:
skiprows5 = 120
maxrows5 = 270

data5 = np.array(read(f'dati_rosso_olio.txt', skiprows5, maxrows5))
popt5, pcov5 = plottiamo_plotly(data5)  # scegliere plotly o pyplot
FWHM(popt5)

FWHM = 269.39


# caso doppio

In [145]:
# gaussiana 1
skiprows6 = 10
maxrows6 = 340
data6 = np.array(read(f'zoom1_sas.txt', skiprows6, maxrows6))

# gaussiana 2
skiprows7 = 350
maxrows7 = 600
data7 = np.array(read(f'zoom1_sas.txt', skiprows7, maxrows7))

# tot dati
datatot = tuple(np.concatenate((a, b)) for a, b in zip(data6, data7))

In [148]:
fig = go.Figure()
X, Y = datatot

# rappresentazione dati
fig.add_trace(go.Scatter(x = X, y = Y, mode = 'lines', line_color = 'plum', 
                         showlegend=False,hoverinfo='skip'))

# rappresentazione fit6
popt6, pcov6 = fittiamo(*data6)
fig.add_trace(go.Scatter(x = data6[0], y = gaussian(data6[0], *popt6), mode='lines',showlegend=False,
    marker = dict(color = 'purple')))

# rappresentazione fit7
popt7, pcov7 = fittiamo(*data7)
fig.add_trace(go.Scatter(x = data7[0], y = gaussian(data7[0], *popt7), mode='lines',showlegend=False,
    marker = dict(color = 'purple')))

fig.update_layout(
        yaxis_title = "Gray Value",
        xaxis_title = "Distance (μm)",
        # title='g<sup>2</sup>(τ) vs t',
        #xaxis = dict(range = [np.log10(0.0001), np.log10(0.010)]),
        #width = 800,
        #height = 600,
        plot_bgcolor = 'white',
        font = dict(
            #family="Courier New, monospace",
            size = 18,
            color = "Black"
        ),
            # yaxis=dict(range=[0, 0.06])
    )

fig.update_xaxes(
    mirror = True,
    ticks = 'outside',
    showline = True,
    linecolor = 'black',
    gridcolor = 'lightgrey'
)
fig.update_yaxes(
    mirror = True,
    ticks = 'outside',
    showline = True,
    linecolor = 'black',
    gridcolor = 'lightgrey'
)
fig.show()
FWHM(popt6)
FWHM(popt7)

FWHM = 314.82
FWHM = 302.32
